In [ ]:
# graphviz is installed already, but need lib too
!apt install libgraphviz-dev
!pip install pygraphviz

import warnings
warnings.filterwarnings(action='ignore')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk librsvg2-common libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libgvc6-plugins-gtk librsvg2-common libxdot4
0 upgraded, 9 newly installed, 0 to remove and 16 not upgraded.
Need to get 2,433 kB of archives.
After this operation, 7,694 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgtk2.0-common all 2.24.33-2ubuntu2 [125 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgtk2.0-0 amd64 2.24.33-2ubuntu2 [2,037 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgail18 amd64 2.24.33-2ubuntu2 [15.9 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Define the generate_data function
def generate_data(data_type, n_samples):
    alpha = 3
    beta1 = 0.5
    beta2 = 0.6
    beta3 = 0.7
    gamma1 = 2
    gamma2 = 3

    # Generate confounders
    X1 = np.random.normal(0, 1, size=n_samples)
    X2 = np.random.normal(0, 1, size=n_samples)
    X3 = np.random.normal(0, 1, size=n_samples)

    if data_type == "data1":
        T = beta1 * X1 +  + beta2 * X2 + beta3 * X3 + np.random.normal(loc=0, scale=1, size=n_samples)
        M = gamma1 * T + beta2 * X2 + np.random.normal(loc=0, scale=0.5, size=n_samples)
        Y = alpha +  gamma2* T + gamma1 * M + beta1* X1 + beta2 * X2 + beta3 * X3 + np.random.normal(loc=0, scale=1, size=n_samples)
        return pd.DataFrame({'Intervention': T, 'Mediator': M, 'X1': X1, 'X2': X2, 'X3': X3, 'Outcome': Y})

    elif data_type == "data2":
        T = beta1 * X1 + + beta2 * X2 + beta3 * X3 + np.random.normal(loc=0, scale=1, size=n_samples)
        Y = alpha  + gamma2* T + beta1* X1 + np.random.normal(loc=0, scale=1, size=n_samples)
        return pd.DataFrame({'Intervention': T, 'X1': X1, 'X2': X2, 'X3': X3, 'Outcome': Y})

    elif data_type == "data3":
        prob_T = 1 / (1 + np.exp(-(X1 + X2)))  # Logistic function
        T = (np.random.rand(n_samples) < prob_T).astype(int)
        M = gamma1 * T + beta1 * X1 - beta2 * X2 + np.random.normal(loc=0, scale=1, size=n_samples)
        Y = alpha + gamma2* T + gamma1 * M + beta1 * X1 + beta2 * X2 + beta3 * X3 + np.random.normal(loc=0, scale=1, size=n_samples)
        return pd.DataFrame({'Intervention': T, 'Mediator': M, 'X1': X1, 'X2': X2, 'X3': X3, 'Outcome': Y})

    elif data_type == "data4":
        prob_T = 1 / (1 + np.exp(-(X1 + X2 + X3)))  # Logistic function
        T = (np.random.rand(n_samples) < prob_T).astype(int)
        Y = alpha + gamma2 * T + beta1 * X1 + beta2 * X2 + np.random.normal(loc=0, scale=1, size=n_samples)
        return pd.DataFrame({'Intervention': T, 'X1': X1, 'X2': X2, 'X3': X3, 'Outcome': Y})

    else:
        raise ValueError("Unknown data type")

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Concatenate

def train_CFNN(data, sample_size):  # Added sample_size as a parameter

    train_data = data.iloc[:sample_size]
    test_data = data.iloc[sample_size:]

    # Extract X and Y from the data
    if 'Mediator' in data.columns:
        X = train_data[['X1', 'X2', 'X3', 'Mediator']].values
    else:
        X = train_data[['X1', 'X2', 'X3']].values

    T = train_data['Intervention'].values.reshape(-1, 1)  # Ensuring T is 2D
    Y = train_data['Outcome'].values

    # Define the CFNN model
    input_layer = Input(shape=(X.shape[1],))
    treatment_layer = Input(shape=(1,))

    concatenated = Concatenate()([input_layer, treatment_layer])

    hidden_layer = Dense(128, activation='relu')(concatenated)  # Changed from input_layer to concatenated
    hidden_layer = Dropout(0.3)(hidden_layer)
    hidden_layer = Dense(128, activation='relu')(hidden_layer)
    hidden_layer = BatchNormalization()(hidden_layer)
    hidden_layer = Dense(64, activation='relu')(hidden_layer)
    hidden_layer = Dropout(0.3)(hidden_layer)

    output_layer = Dense(1)(hidden_layer)
    model = Model(inputs=[input_layer, treatment_layer], outputs=output_layer)

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')

    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

    model.fit([X, T], Y, epochs=50, batch_size=32, verbose=0, validation_split=0.2, callbacks=[early_stop, reduce_lr])  # Changed from [X] to [X, T]

    if 'Mediator' in data.columns:
        X_test = test_data[['X1', 'X2', 'X3', 'Mediator']].values
    else:
        X_test = test_data[['X1', 'X2', 'X3']].values

    T_test = test_data['Intervention'].values.reshape(-1, 1)  # Reshape T_test
    Y_test = test_data['Outcome'].values

    if 'data1' in data_type or 'data2' in data_type:
        # Continuous treatment
        T_test_tensor = tf.convert_to_tensor(T_test, dtype=tf.float32)  # Convert to tensor
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(T_test_tensor)
            predictions = model([X_test, T_test_tensor])
        gradients = tape.gradient(predictions, T_test_tensor)
        causal_effect = np.mean(gradients.numpy())
        Y_pred = predictions.numpy().squeeze()


    else:
        # Binary treatment
        T_test_1 = np.ones_like(T_test)
        T_test_0 = np.zeros_like(T_test)

        Y_pred_1 = model.predict([X_test, T_test_1])
        Y_pred_0 = model.predict([X_test, T_test_0])

        causal_effect = np.mean(Y_pred_1 - Y_pred_0)
        Y_pred = model.predict([X_test, T_test]).squeeze()

    # Calculate MAPE
    mape = np.mean(np.abs((Y_test - Y_pred) / Y_test))*100
    rmse = np.sqrt(np.mean(np.square((Y_test - Y_pred))))

    return mape, rmse, causal_effect


data_types = ["data1", "data2", "data3", "data4"]
sample_sizes = [50, 100, 1000]

results = []

for data_type in data_types:
    for sample_size in sample_sizes:
        for iteration in range(100):  # Train for 100 iterations (as mentioned in the comment)
            data = generate_data(data_type, sample_size+1000)
            mape, rmse, causal_effect = train_CFNN(data, sample_size)


            results.append({
                'Data_Type': data_type,
                'Sample_Size': sample_size,
                'Iteration': iteration + 1,
                'MAPE': mape,
                'RMSE' : rmse,
                'Causal_Effect': causal_effect,  # You can add this back once you compute causal_effect
                'Method': 'CFNN'
            })


results_df = pd.DataFrame(results)

print(results_df)


32/32 [==============================] - 0s 2ms/step
     Data_Type  Sample_Size  Iteration        MAPE      RMSE  Causal_Effect  \
0        data1           50          1   91.733139  7.559428       0.571090   
1        data1           50          2   78.210107  5.732518       0.970891   
2        data1           50          3  102.023233  7.953526       0.217713   
3        data1           50          4   72.826845  7.382050       0.500615   
4        data1           50          5   96.538996  7.121534       0.691907   
...        ...          ...        ...         ...       ...            ...   
1195     data4         1000         96   38.289230  1.020308       2.854106   
1196     data4         1000         97   66.309689  1.059174       2.886820   
1197     data4         1000         98   62.637151  1.053114       2.561312   
1198     data4         1000         99   36.591143  1.044405       2.930557   
1199     data4         1000        100   40.318818  1.040800       2.868540   

In [ ]:
results_df.to_csv('/content/drive/MyDrive/Colab Notebooks/results_cfnn3.csv', index=False)

In [ ]:
results_df.to_csv('results_cfnn2.csv', index=False)

In [ ]:
results_df_g=results_df.groupby(['Data_Type', 'Sample_Size']).mean().reset_index()
results_df_g

In [ ]:
results_df_g=results_df.groupby(['Data_Type', 'Sample_Size']).mean().reset_index()
results_df_g

KeyError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

import numpy as np
import tensorflow as tf


def create_transt_model():
    input_x1 = tf.keras.layers.Input(shape=(1,), name="X1")
    input_x2 = tf.keras.layers.Input(shape=(1,), name="X2")
    input_x3 = tf.keras.layers.Input(shape=(1,), name="X3")
    input_med = tf.keras.layers.Input(shape=(1,), name="Mediator")
    input_interv = tf.keras.layers.Input(shape=(1,), name="Intervention")

    # Concatenate inputs
    merged = tf.keras.layers.Concatenate(axis=-1)([input_x1, input_x2, input_x3, input_med, input_interv])

    # Dense layers
    dense1 = tf.keras.layers.Dense(128, activation='relu')(merged)
    dense2 = tf.keras.layers.Dense(64, activation='relu')(dense1)

    # Predict outcome
    outcome = tf.keras.layers.Dense(1, activation='linear', name="Outcome")(dense2)

    model = tf.keras.models.Model(inputs=[input_x1, input_x2, input_x3, input_med, input_interv], outputs=outcome)
    model.compile(optimizer='adam', loss='mse')  # Using MSE for simplicity

    return model

tranet_model = create_transt_model()


data_types = ["data1", "data2", "data3", "data4"]
sample_sizes = [50, 100, 1000]

results = []

for data_type in data_types:
    for sample_size in sample_sizes:
        for iteration in range(100):
            data = generate_data(data_type, sample_size+1000)

            # Determine which features are available based on the data type
            if data_type in ["data1", "data3"]:
                features = ['X1', 'X2', 'X3', 'Mediator', 'Intervention']
            else:
                features = ['X1', 'X2', 'X3', 'Intervention']

            # Split the data for training and testing
            train_size = int(0.8 * len(data['Outcome']))
            X_train = [data[feature][:train_size] for feature in features]
            X_test = [data[feature][train_size:] for feature in features]
            Y_train = data['Outcome'][:train_size]
            Y_test = data['Outcome'][train_size:]

            # Train the Tranet model
            tranet_model.fit(X_train, Y_train, epochs=10)

            # Predict using the model
            Y_pred = tranet_model.predict(X_test)

            # Calculate MAPE and RMSE
            mape = np.mean(np.abs((Y_test - Y_pred.reshape(-1,)) / Y_test)) * 100
            rmse = np.sqrt(np.mean(np.square((Y_test - Y_pred.reshape(-1,)))))

            # Append the results
            results.append({
                'Data_Type': data_type,
                'Sample_Size': sample_size,
                'Iteration': iteration + 1,
                'MAPE': mape,
                'RMSE': rmse,
                'Method': 'Tranet'
            })

results_df = pd.DataFrame(results)
print(results_df)



In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import regularizers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler


def make_tarnet(input_dim, reg_l2=0.01):
    '''
    The first argument is the column dimension of our data.
    It needs to be specified because the functional API creates a static computational graph
    The second argument is the strength of regularization we'll apply to the output layers
    '''
    x = Input(shape=(input_dim,), name='input')

    # REPRESENTATION
    #in TF2/Keras it is idiomatic to instantiate a layer and pass its inputs on the same line unless the layer will be reused
    #Note that we apply no regularization to the representation layers
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(x)
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_2')(phi)
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_3')(phi)

    # HYPOTHESIS
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)

    # second layer
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_2')(y0_hidden)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_2')(y1_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)

    #a convenience "layer" that concatenates arrays as columns in a matrix
    concat_pred = Concatenate(1)([y0_predictions, y1_predictions])
    #the declarations above have specified the computational graph of our network, now we instantiate it
    model = Model(inputs=x, outputs=concat_pred)

    return model


results = []

for data_type in data_types:
    for sample_size in sample_sizes:
        for iteration in range(100):
            data = generate_data(data_type, sample_size+1000)

            if data_type in ["data1", "data3"]:
                features = ['X1', 'X2', 'X3', 'Mediator']
            else:
                features = ['X1', 'X2', 'X3']

            if data_type in ["data1", "data2"]:
                # Continuous treatment
                features.append('Intervention')

            X_train = data[features][:train_size].values
            X_test = data[features][train_size:].values
            Y_train = data['Outcome'][:train_size].values
            T_train = data['Intervention'][:train_size].values
            Y_test = data['Outcome'][train_size:].values
            T_test = data['Intervention'][train_size:].values

            if data_type in ["data1", "data2"]:
                yt_train = Y_train
            else:
                T_train = data['Intervention'][:train_size].values
                yt_train = np.concatenate([Y_train[T_train == 0][:, np.newaxis], Y_train[T_train == 1][:, np.newaxis]], axis=1)


            tarnet_model = make_tarnet(input_dim=len(features))
            tarnet_model.compile(optimizer=Adam(0.001), loss='mse')
            tarnet_model.fit(X_train, yt_train, epochs=10, batch_size=64, validation_split=0.1, verbose=0)


            Y_pred = tarnet_model.predict(X_test)

            if data_type in ["data1", "data2"]:
                # For continuous treatments, taking the average of the two outputs as the prediction
                Y_pred_avg = np.mean(Y_pred, axis=1)
                mape = np.mean(np.abs((Y_test - Y_pred_avg) / Y_test)) * 100
                rmse = np.sqrt(np.mean(np.square(Y_test - Y_pred_avg)))
                causal_effects = Y_pred_avg - Y_test  # adjust as needed
            else:
                Y_pred_control, Y_pred_treated = Y_pred[:, 0], Y_pred[:, 1]
                mape = np.mean(np.abs((Y_test - Y_pred_control) / Y_test)) * 100
                rmse = np.sqrt(np.mean(np.square(Y_test - Y_pred_control)))
                causal_effects = Y_pred_treated - Y_pred_control


            results.append({
                'Data_Type': data_type,
                'Sample_Size': sample_size,
                'Iteration': iteration + 1,
                'MAPE': mape,
                'RMSE': rmse,
                'Causal_effect': causal_effects,
                'Method': 'TARNET'
            })

results_df = pd.DataFrame(results)
print(results_df)

In [ ]:
results_df.to_csv('results_tranet.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
results_df.to_csv('/content/drive/My Drive/Colab Notebooks/results_tarnet.csv', index=False)


In [ ]:
Y_pred

In [ ]:
results_df

In [ ]:
causal_effects = Y_pred_treated - Y_pred_control


In [ ]:
avg_causal_effect = np.mean(causal_effects)

In [ ]:
avg_causal_effect